Vertical scaling in Kubernetes terms means we can use only one container or one pod. In that, we can choose to run Julia either with multiple threads or multiple processes or even in hybrid mode with multiple processes, each with multiple threads.

Connect to the kubernetes cluster.

In [1]:
using Kuber
ctx = KuberContext();
Kuber.set_api_versions!(ctx);

┌ Info: unsupported crd.projectcalico.org/v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:131


## Multithreading

Start julia with JULIA_NUM_THREADS environment set. Notice the `env` section where we have added the environment variable specifying the number of threas Julia should run with.

We are also allocating 4 CPUs to the pod, corresponding to the number of threads we want to run.

And we are verifying that in the Julia process by printing out the number of threads.

In [2]:
julia_pod = kuber_obj(ctx, """{
           "kind": "Pod",
           "metadata":{
               "name": "julia-pod",
               "namespace": "default",
               "labels": {
                   "name": "julia-pod"
               }
           },
           "spec": {
               "containers": [{
                   "name": "julia",
                   "image": "julia:1.4.1",
                   "env": [{
                       "name": "JULIA_NUM_THREADS",
                       "value": "4"
                   }],
                   "command": ["julia"],
                   "args": [
                       "-e",
                       "while true; println(Threads.nthreads()); sleep(5); end"
                   ],
                   "resources": {
                       "limits": {
                           "cpu": "4"
                       },
                       "requests": {
                           "cpu": "4"
                       }
                   }
               }]
           }
       }""");

In [3]:
pod = put!(ctx, julia_pod);

In [4]:
;kubectl logs julia-pod

4
4


In [5]:
delete!(ctx, pod);

## Multiprocerssing

Use Julia Distributed package.

We are requesting for 4 CPUs to be allocated to the pod. And we are starting 4 worker processes in our Julia code. And we are verifying that by printing our the number of workers.

Actually there would be 5 processes - including the master julia process. But if our model is that the workers are doing most of the CPU intensive work and the master is only coordinating, this is not a problem.

In [6]:
julia_pod = kuber_obj(ctx, """{
           "kind": "Pod",
           "metadata":{
               "name": "julia-pod",
               "namespace": "default",
               "labels": {
                   "name": "julia-pod"
               }
           },
           "spec": {
               "containers": [{
                   "name": "julia",
                   "image": "julia:1.4.1",
                   "command": ["julia"],
                   "args": [
                       "-e",
                       "using Distributed; addprocs(4); while true; println(nworkers()); sleep(5); end"
                   ],
                   "resources": {
                       "limits": {
                           "cpu": "4"
                       },
                       "requests": {
                           "cpu": "4"
                       }
                   }
               }]
           }
       }""");

In [7]:
pod = put!(ctx, julia_pod);

In [8]:
;kubectl logs julia-pod

4
4


In [9]:
delete!(ctx, pod);